In [47]:
# Importing all necessary libraries

import numpy as np

import pandas as pd

In [48]:
# Loading in the daily and monthly crsp datasets

crsp_daily = pd.read_feather('~/FIN_585/crsp_data/crsp_daily.ftr')

crsp_monthly = pd.read_feather('~/FIN_585/crsp_data/crsp_monthly.ftr')

In [49]:
# Cleaning the daily dataset

crsp_daily['prc'] = abs(crsp_daily['prc'])

crsp_daily['prc_lag'] = crsp_daily.groupby('permno')['prc'].shift(1)

crsp_daily = crsp_daily[crsp_daily['ret'] > -1]

crsp_daily = crsp_daily[crsp_daily['prc_lag'] > 5]

crsp_daily.sort_values(by = ['permno', 'caldt'], inplace = True)

crsp_daily.drop(columns = ['shrcd', 'excd', 'siccd', 'vol', 'shr', 'prc_lag'], inplace = True)

In [50]:
# Adding column for positive and negative returns

crsp_daily['ret_class'] = np.where(crsp_daily['ret'].shift(1) >= 0, '1', '0')

In [51]:
# Calculate rolling yearly number of positive and negative days for each stock

n = 252

crsp_daily['pos_days'] = crsp_daily.groupby('permno').rolling(window = n, min_periods = n)['ret_class'].sum().reset_index(level=0, drop=True)

crsp_daily['neg_days'] = n - crsp_daily['pos_days']

crsp_daily['%pos'] = crsp_daily['pos_days'] / n

crsp_daily['%neg'] = crsp_daily['neg_days'] / n

crsp_daily['%neg - %pos'] = crsp_daily['%neg'] - crsp_daily['%pos']

crsp_daily.drop(columns = ['ret_class', 'pos_days', 'neg_days', '%pos', '%neg', 'prc', 'ret'], inplace = True)

crsp_daily.dropna(inplace = True)

In [52]:
# Getting daily data ready for merging with monthly data

crsp_daily_resampled = crsp_daily.set_index('caldt').groupby('permno').resample('ME').first().droplevel('permno').reset_index()

In [53]:
# Cleaning the monthly dataset

crsp_monthly.drop(columns = ['shrcd', 'excd', 'siccd', 'vol', 'shr', 'cusip', 'ticker', 'prc', 'cumfacshr'], inplace = True)

crsp_monthly.dropna(inplace = True)

In [54]:
# Merging the daily and monthly datasets

merged_data = pd.merge(crsp_monthly, crsp_daily, on = ['permno', 'caldt'], how = 'inner')

merged_data.dropna(inplace = True)

In [55]:
# Calculating momentum

merged_data['log_ret'] = np.log(1 + merged_data['ret'])

merged_data['cum_log_ret'] = merged_data.groupby('permno')['log_ret'].rolling(window = 11, min_periods = 11).sum().reset_index(drop=True)

merged_data['momentum'] = merged_data.groupby('permno')['cum_log_ret'].shift(2)

merged_data.drop(columns = ['log_ret', 'cum_log_ret'], inplace = True)

merged_data.dropna(inplace = True)

In [59]:
# Calculate information discreatness 'id'

merged_data['id'] = merged_data['momentum'] * merged_data['%neg - %pos']

In [61]:
# Unconditional double sort portfolios by momentum and id

merged_data['momentum_bins'] = merged_data.groupby('caldt')['momentum'].transform(lambda x: pd.qcut(x, 2, labels = False))

merged_data['id_bins'] = merged_data.groupby('caldt')['id'].transform(lambda x: pd.qcut(x, 5, labels = False))

port = merged_data.groupby(['caldt', 'momentum_bins', 'id_bins'])['ret'].mean().unstack(level=['momentum_bins', 'id_bins'])

In [140]:
(port.describe()*100).round(2)

momentum_bins         0                                              \
id_bins               0          1          2          3          4   
count          96800.00  113400.00  115400.00  115400.00  115400.00   
mean               0.84       1.01       1.19       1.15       1.79   
std               10.86       7.39       6.71       6.90       6.92   
min              -35.98     -35.42     -37.27     -35.79     -29.59   
25%               -3.82      -1.82      -1.61      -1.84      -1.50   
50%                0.67       1.28       1.47       1.16       1.51   
75%                4.59       3.78       3.91       4.01       4.71   
max              116.67      70.76      69.62      64.68      71.15   

momentum_bins          1                                             
id_bins                0          1          2          3         4  
count          115400.00  115400.00  115400.00  110100.00  92000.00  
mean                2.36       1.51       1.04       0.20     -0.64  
std                 6.13       6.00       6.23       6.43      9.24  
min               -27.40     -28.76     -27.28     -30.68    -38.00  
25%                -0.69      -1.13      -2.07      -3.15     -5.54  
50%                 2.73       1.99       1.35       0.36     -0.89  
75%                 5.57       4.46       4.10       3.84      4.01  
max                59.93      50.56      51.91      34.76     69.20

In [117]:
# Calculating differences in high and low momentum across id bins

winner_losser_port = port[1] - port[0]

winner_losser_port['spread'] = winner_losser_port[0] - winner_losser_port[4]

In [167]:
(winner_losser_port.describe()*100).round(2)

id_bins,0,1,2,3,4,spread
count,96800.00,113400.00,115400.00,110100.00,92000.00,85800.00
mean,1.42,0.49,-0.16,-0.93,-2.16,3.80
std,8.33,4.03,2.63,3.72,7.36,9.13
min,-81.55,-24.20,-28.98,-38.63,-41.96,-62.71
25%,-2.16,-1.32,-1.37,-2.52,-5.19,-0.38
50%,1.83,0.61,-0.07,-0.83,-2.20,3.76
75%,5.67,2.57,1.10,0.69,0.71,8.08
max,27.47,17.23,15.38,17.17,64.96,47.14
